In [1]:
import pandas as pd
import seaborn as sns

In [2]:
df=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv')
df


,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,17.166667,55.200000,7.026667,84.256667,17.200000,41.626667,18.2000,48.900000,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,17.166667,55.200000,6.833333,84.063333,17.200000,41.560000,18.2000,48.863333,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,17.166667,55.090000,6.560000,83.156667,17.200000,41.433333,18.2000,48.730000,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,17.166667,55.090000,6.433333,83.423333,17.133333,41.290000,18.1000,48.590000,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,17.200000,55.090000,6.366667,84.893333,17.200000,41.230000,18.1000,48.590000,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,2016-05-27 17:20:00,100,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,23.200000,52.400000,24.796667,1.000000,24.500000,44.500000,24.7000,50.074000,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812,43.096812
19731,2016-05-27 17:30:00,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,23.230000,52.326667,24.196667,1.000000,24.557143,44.414286,24.7000,49.790000,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940,49.282940
19732,2016-05-27 17:40:00,270,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,23.230000,52.266667,23.626667,1.000000,24.540000,44.400000,24.7000,49.660000,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117,29.199117
19733,2016-05-27 17:50:00,420,10,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,23.200000,52.200000,22.433333,1.000000,24.500000,44.295714,24.6625,49.518750,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333,6.322784,6.322784


In [3]:
simple_linear_model_feature_df = df[['T2']]
simple_linear_model_target = df['T6']

In [4]:
from sklearn.model_selection import train_test_split
simple_x_train, simple_x_test, simple_y_train, simple_y_test = train_test_split(simple_linear_model_feature_df, simple_linear_model_target, test_size=0.3, random_state=42)

In [5]:
from sklearn.linear_model import LinearRegression
simple_linear_model = LinearRegression()
simple_linear_model.fit(simple_x_train, simple_y_train)

LinearRegression()

In [6]:
simple_linear_model_prediction = simple_linear_model.predict(simple_x_test)

In [7]:
from sklearn.metrics import r2_score
simple_linear_model_r2_score = r2_score(simple_y_test, simple_linear_model_prediction)
round(simple_linear_model_r2_score, 2)

0.64

In [8]:
multiple_reg_df = df.drop(columns=['date', 'lights'])

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(multiple_reg_df), columns=multiple_reg_df.columns)

In [10]:
multiple_reg_features_df = normalised_df.drop(columns=['Appliances'])
multiple_reg_target = normalised_df['Appliances']

In [11]:
x_train, x_test, y_train, y_test = train_test_split(multiple_reg_features_df, multiple_reg_target, test_size=0.3, random_state=42)

In [12]:
multiple_reg_model = LinearRegression()
multiple_reg_model.fit(x_train, y_train)

LinearRegression()

In [13]:
multiple_reg_prediction = multiple_reg_model.predict(x_test)

In [14]:
from sklearn.metrics import mean_absolute_error
multiple_reg_mae = mean_absolute_error(y_test, multiple_reg_prediction)
round(multiple_reg_mae, 2)

0.05

In [15]:
import numpy as np
multiple_reg_rss = np.sum(np.square(y_test - multiple_reg_prediction))
round(multiple_reg_rss, 2)

45.35

In [16]:
from sklearn.metrics import mean_squared_error
multiple_reg_rmse = np.sqrt(mean_squared_error(y_test, multiple_reg_prediction))
round(multiple_reg_rmse, 3)

0.088

In [17]:
from sklearn.metrics import r2_score
multiple_reg_coeff_of_determination = r2_score(y_test, multiple_reg_prediction)
round(multiple_reg_coeff_of_determination, 2)

0.15

In [18]:
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

In [19]:
multiple_reg_weights = get_weights_df(multiple_reg_model, x_train, 'Multiple Linear Regression Model Weight')
multiple_reg_weights.sort_values(by='Multiple Linear Regression Model Weight')

,Features,Multiple Linear Regression Model Weight
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


In [20]:
from sklearn.linear_model import Ridge
ridge_reg_model = Ridge(alpha=0.4)
ridge_reg_model.fit(x_train, y_train)

Ridge(alpha=0.4)

In [21]:
ridge_reg_prediction = ridge_reg_model.predict(x_test)

In [22]:
ridge_reg_rmse = np.sqrt(mean_squared_error(y_test, ridge_reg_prediction))
round(ridge_reg_rmse, 3)

0.088

In [23]:
from sklearn.linear_model import Lasso
lasso_reg_model = Lasso(alpha=0.001)
lasso_reg_model.fit(x_train, y_train)

Lasso(alpha=0.001)

In [24]:
lasso_reg_prediction = lasso_reg_model.predict(x_test)

In [25]:
lasso_reg_weights = get_weights_df(lasso_reg_model, x_train, 'Lasso Regression Model Weight')
lasso_reg_weights

,Features,Lasso Regression Model Weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


In [26]:
lasso_reg_weights[lasso_reg_weights['Lasso Regression Model Weight']!=0]

,Features,Lasso Regression Model Weight
0,RH_out,-0.049557
1,RH_8,-0.000110
24,Windspeed,0.002912
25,RH_1,0.017880


In [27]:
lasso_reg_rmse = np.sqrt(mean_squared_error(y_test, lasso_reg_prediction))
round(lasso_reg_rmse, 3)

0.094